# License

> Copyright (c) 2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
>
> Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
>
> http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Introduction

In this tutorial we show how use NeMo **neural audio codecs** at inference time. To learn more about training and finetuning neural audio codecs in NeMo, check the [Audio Codec Training tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/Audio_Codec_Training.ipynb).

An audio codec typically consists of an encoder, a quantizer and a decoder, with a typical architecture depicted in the figure below.
An audio codec can be used to encode an input audio signal into a sequence of discrete values.
In this tutorial, the discrete values will be referred to as **audio tokens**.
The obtained audio tokens can be decoded into an output audio signal.

Audio tokens can be used to represent the input audio for an automatic speech recognition (ASR) model [[1](https://arxiv.org/abs/2309.10922), [2](https://arxiv.org/pdf/2407.03495)], or to represent the output audio of a text-to-speech (TTS) system [[3](https://arxiv.org/abs/2406.05298), [4](https://arxiv.org/pdf/2406.17957)].

NeMo provides several neural audio codec models, inlcuding audio codecs and mel codecs at different sampling rates.
The list of the available models can be found [here](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/tts/checkpoints.html#codec-models).

<div>
<img src="https://github.com/NVIDIA/NeMo/releases/download/v1.22.0/nemo_audio_codec.png" width="800", height="400"/>
</div>

# Setup

In [ ]:
BRANCH = 'main'
# Install NeMo library. If you are running locally (rather than on Google Colab), follow the instructions at https://github.com/NVIDIA/NeMo#Installation

if 'google.colab' in str(get_ipython()):
    !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
import math
import wget
import os
import librosa
import torch
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
from pathlib import Path


# Utility for displaying signals and metrics
def show_signal(signal: np.ndarray, sample_rate: int = 16000, tag: str = 'Signal'):
    """Show the time-domain signal and its spectrogram.
    """
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 2.5))

    # show waveform
    t = np.arange(0, len(signal)) / sample_rate

    ax[0].plot(t, signal)
    ax[0].set_xlim(0, t.max())
    ax[0].grid()
    ax[0].set_xlabel('time / s')
    ax[0].set_ylabel('amplitude')
    ax[0].set_title(tag)

    n_fft = 1024
    hop_length = 256

    D = librosa.amplitude_to_db(np.abs(librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)), ref=np.max)
    img = librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=sample_rate, n_fft=n_fft, hop_length=hop_length, ax=ax[1])
    ax[1].set_title(tag)

    plt.tight_layout()
    plt.colorbar(img, format="%+2.f dB", ax=ax)


# Utility for displaying a latent representation
def show_latent(latent: np.ndarray, tag: str):
    plt.figure(figsize = (16, 3))
    img = plt.imshow(latent, aspect='equal')
    plt.colorbar(img, ax=plt.gca())
    plt.title(tag)
    plt.xlabel('Time frame')
    plt.ylabel('Latent vector index')
    plt.tight_layout()

In [ ]:
# Working directory
ROOT_DIR = Path().absolute() / 'codec_tutorial'

# Create dataset directory
DATA_DIR = ROOT_DIR / 'data'
DATA_DIR.mkdir(parents=True, exist_ok=True)

audio_path = DATA_DIR / 'LJ023-0089.wav'
audio_url = "https://multilangaudiosamples.s3.us-east-2.amazonaws.com/LJ023-0089.wav"

if not os.path.exists(audio_path):
  wget.download(audio_url, audio_path.as_posix())

# Load a model from NGC

Any of the [pretrained checkpoints](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/tts/checkpoints.html#codec-models) could be used for inference.
Here, we use `mel_codec_22khz_fullband_medium`, which works for 22.05 kHz audio signals.

The model can be easily restored from NGC:

In [ ]:
from nemo.collections.tts.models.audio_codec import AudioCodecModel

# Optionally specify a pretrained model to fine-tune from. To train from scratch, set this to 'None'.
model_name = 'mel_codec_22khz_fullband_medium'
codec_model = AudioCodecModel.from_pretrained(model_name)
codec_model.freeze()

Show information about the loaded model:

In [ ]:
print(f'Loaded model from NeMo:')
print(f'\tmodel name       : {model_name}')
print(f'\tsample rate      : {codec_model.sample_rate} Hz')
print(f'\tlatent dimension : {codec_model.vector_quantizer.codebook_dim}')

print('\n\nModel summary:')
print(codec_model.summarize())

# Inference

## Processing audio

Here we use the codec model to process the input audio by applying the complete model. The input signal is encoded, quantized, dequantized and decoded. Finally, a reconstructed signal is obtained.

In [ ]:
input_audio, sr = librosa.load(audio_path, sr=codec_model.sample_rate)

# Shape (batch, time)
input_audio_tensor = torch.from_numpy(input_audio).unsqueeze(dim=0).to(codec_model.device)

# Shape (batch,)
input_audio_len = torch.tensor([input_audio_tensor.size(-1)]).to(codec_model.device)

# Process audio using the codec model
output_audio_tensor, _ = codec_model(audio=input_audio_tensor, audio_len=input_audio_len)

# Output audio
output_audio = output_audio_tensor.squeeze().cpu().numpy()

# Show signals
show_signal(input_audio, tag='Input audio', sample_rate=codec_model.sample_rate)
show_signal(output_audio, tag='Output audio', sample_rate=codec_model.sample_rate)

# Play audio
print('Input audio')
ipd.display(ipd.Audio(input_audio, rate=codec_model.sample_rate))

print('Output audio')
ipd.display(ipd.Audio(output_audio, rate=codec_model.sample_rate))

## Audio tokens

Audio tokens can be easily computed by using the `encode` method of the `AudioCodec` model.

In [ ]:
# Convert audio to tokens
tokens, tokens_len = codec_model.encode(audio=input_audio_tensor, audio_len=input_audio_len)

print('tokens information:')
print(f'\tshape (batch, codebook, time frame) : {tokens.size()}')
print(f'\tdtype                               : {tokens.dtype}')
print(f'\tmin                                 : {tokens.min()}')
print(f'\tmax                                 : {tokens.max()}')

# Number of codebooks should match the number of codebooks/groups
if hasattr(codec_model.vector_quantizer, 'num_groups'):
    # Group FSQ
    assert tokens.size(1) == codec_model.vector_quantizer.num_groups
    print(f'\tnum_groups                          : {tokens.size(1)}')
elif hasattr(codec_model.vector_quantizer, 'codebooks'):
    # RVQ
    assert tokens.size(1) == len(codec_model.vector_quantizer.codebooks)
    print(f'\tnum_codebooks                       : {tokens.size(1)}')

Similarly, audio can be easily reconstructed from audio tokens using the `decode` method of the `AudioCodec` models.

In [ ]:
# Convert tokens back to audio
output_audio_from_tokens_tensor, _ = codec_model.decode(tokens=tokens, tokens_len=tokens_len)
output_audio_from_tokens = output_audio_from_tokens_tensor.squeeze().cpu().numpy()

# Show signals
show_signal(output_audio_from_tokens, tag='Output audio from tokens', sample_rate=codec_model.sample_rate)
show_signal(output_audio_from_tokens - output_audio, tag='Difference compared to forward pass', sample_rate=codec_model.sample_rate)

## Latent representation

Continuous (non-discrete) latent representation at the output of the encoder can be easily computed using the `encode_audio` method of the `AudioCodec` model.

In [ ]:
# Convert audio to the encoded representation
encoded, encoded_len = codec_model.encode_audio(audio=input_audio_tensor, audio_len=input_audio_len)

print('encoded information:')
print(f'\tshape (batch, codebook, time frame) : {encoded.size()}')
print(f'\tdtype                               : {encoded.dtype}')
print(f'\tmin                                 : {encoded.min()}')
print(f'\tmax                                 : {encoded.max()}')


# Show the encoded representation
show_latent(encoded.squeeze().cpu().numpy(), tag='Encoder output')

The encoded representation can be easily converted to tokens, dequantized into a continuous latent representation and decoded back to audio.

In [ ]:
# Encoder output to tokens
tokens = codec_model.quantize(encoded=encoded, encoded_len=encoded_len)

# Tokens back to a continuous vector
dequantized = codec_model.dequantize(tokens=tokens, tokens_len=encoded_len)

# Reconstruct audio
output_audio_from_latent_tensor, _ = codec_model.decode_audio(inputs=dequantized, input_len=encoded_len)
output_audio_from_latent = output_audio_from_latent_tensor.squeeze().cpu().numpy()

# Show dequantized latent representation
show_latent(dequantized.squeeze().cpu().numpy(), tag='Decoder input')

# Show signals
show_signal(output_audio_from_latent, tag='Output audio from latent', sample_rate=codec_model.sample_rate)
show_signal(output_audio_from_latent - output_audio, tag='Difference compared to forward pass', sample_rate=codec_model.sample_rate)

# Related information

To learn more about audio codec models in NeMo, look at our [documentation](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/tts/models.html#codecs).

For more information on training and finetuning neural audio codecs in NeMo, check the [Audio Codec Training tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/Audio_Codec_Training.ipynb).

# References

1.   [Discrete Audio Representation as an Alternative to Mel-Spectrograms for Speaker and Speech Recognition](https://arxiv.org/abs/2309.10922)
2.   [Codec-ASR: Training Performant Automatic Speech Recognition Systems with Discrete Speech Representations](https://arxiv.org/pdf/2407.03495)
3.   [Spectral Codecs: Spectrogram-Based Audio Codecs for High Quality Speech Synthesis](https://arxiv.org/abs/2406.05298)
4.   [Improving Robustness of LLM-based Speech Synthesis by Learning Monotonic Alignment](https://arxiv.org/pdf/2406.17957)